This program creates a convolutional neural network for image recognition


In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import os
import cv2
from google.colab import drive
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from PIL import Image
import io
import zipfile
from google.colab import drive

# Importing Dataset

Link to dataset: https://drive.google.com/drive/folders/1YPuVjC1qK1inVBkVh5teQRVcT8O9wWBH?usp=sharing

You must add a shortcut of Archive.zip to your own drive. This can be done by hitting the "Dataset" dropdown -> Organize -> Add Shortcut -> My Drive. After the shortcut is added, the following imports will work correctly.

In [16]:
drive.mount('/content/drive/')

# Unzip dataset
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Dataset/dataset.zip", 'r')
zip_ref.extractall("/tmp/dataset")
zip_ref.close()

# Initialize empty lists
X_original = []
Y = []

map_labels ={'buildings' : 0, 'forest' : 1, 'glacier' : 2, 'mountain' : 3, 'sea' : 4, 'street' : 5}

# Define folder path
folder_path = '/tmp/dataset/dataset'

# Check if the folder exists
if os.path.exists(folder_path):
    # Change the current working directory to the "tmp" folder
    os.chdir(folder_path)
else:
    print("The 'tmp' folder does not exist in your Google Drive.")

# Iterate through main folder to define class labels
for label in os.listdir(folder_path):
    label_path = os.path.join(folder_path, label)

    # Iterate through each subfolder and apply class labels
    if os.path.isdir(label_path):
        for image_file in os.listdir(label_path):

            image_path = os.path.join(label_path, image_file)

            img = np.array(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))

            # Reject any images not 150x150 px
            if len(img) != 150: continue

            X_original.append(img)

            # Append the corresponding label (folder name) to Y_train list
            Y.append(map_labels[label])

# Convert to numpy arrays
X_original = np.array(X_original)
Y = np.array(Y)

#Split data into Train and Test data
X_train, X_test, Y_train, Y_test = train_test_split(X_original, Y, test_size = .2, random_state=40)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Create Model and Test Model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape = (150, 150, 3)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(8))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, epochs=10,
                    validation_data=(X_test, Y_test))

Epoch 1/10
 68/424 [===>..........................] - ETA: 8:57 - loss: 9.8799 - accuracy: 0.2316